In [15]:
pip install nltk pyodbc sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [16]:
import pandas as pd
import pyodbc
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [25]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\gaurav\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [29]:
# Defining function to fetch data from SQL database
def fetch_data_from_sql():
    conn_str=(
        "Driver={SQL Server};"
        "Server=DESKTOP-PH9UC50\SQLEXPRESS;"
        "Database=PortfolioProject_MarketingAnalytics;"
        "Trusted_Connection=yes;"
    )
    
    # Establish the connection to the database
    conn=pyodbc.connect(conn_str)

    query="SELECT ReviewID, CustomerID, ProductID, ReviewDate, Rating, ReviewText FROM fact_customer_reviews"

    df=pd.read_sql(query, conn)

    conn.close()

    return df

In [30]:
customer_reviews_df=fetch_data_from_sql()

C:\Users\gaurav\AppData\Local\Temp\ipykernel_18580\392196729.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql(query, conn)


In [32]:
customer_reviews_df

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText
0,1,77,18,2023-12-23,3,"Average experience, nothing special."
1,2,80,19,2024-12-25,5,The quality is top-notch.
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper."
4,5,64,2,2023-07-16,3,"Average experience, nothing special."
...,...,...,...,...,...,...
1358,1359,28,4,2023-05-25,3,Not worth the money.
1359,1360,58,12,2023-11-13,2,"Average experience, nothing special."
1360,1361,96,15,2023-03-07,5,Customer support was very helpful.
1361,1362,99,2,2025-12-03,1,Product did not meet my expectations.


In [33]:
sia=SentimentIntensityAnalyzer()

In [34]:
# Define a function to calculate sentiment scores using VADER
def calculate_sentiment(review):
    sentiment=sia.polarity_scores(review)
    return sentiment['compound']

In [35]:
# Define a function to categorise sentiment using both the sentiment score and the review rating
def categorize_sentiment(score, rating):
    if score>0.05:
        if rating>=4:
            return 'Positive'
        elif rating==3:
            return 'Mixed Positive'
        else:
            return 'Mixed Negative'
    elif score<-0.05:
        if rating<=2:
            return 'Negative'
        elif rating==3:
            return 'Mixed Negative'
        else:
            return 'Mixed Positive'
    else:
        if rating>=4:
            return 'Positive'
        elif rating<=2:
            return 'Negative'
        else:
            return 'Neutral'

In [36]:
# Define a function to bucket sentiment scores into text ranges
def sentiment_bucket(score):
    if score>=0.5:
        return '0.5 to 1.0'
    elif 0.0<=score<0.5:
        return '0.0 to 0.49'
    elif -0.5<=score<0.0:
        return '-0.49 to 0.0'
    else:
        return '-1.0 to -0.5'

In [37]:
# Apply Sentiment analysis to calculate sentiment scores for each review
customer_reviews_df['SentimentScore']=customer_reviews_df['ReviewText'].apply(calculate_sentiment)

In [38]:
# Apply Sentiment categorization using both rext and rating
customer_reviews_df['SentimentCategory']=customer_reviews_df.apply(lambda row: categorize_sentiment(row['SentimentScore'], row['Rating']), axis=1)

In [39]:
# Apply sentiment bucketing to categorize scores into defined ranges
customer_reviews_df['SentimentBucket']=customer_reviews_df['SentimentScore'].apply(sentiment_bucket)

In [42]:
print(customer_reviews_df.head())

   ReviewID  CustomerID  ProductID  ReviewDate  Rating  \
0         1          77         18  2023-12-23       3   
1         2          80         19  2024-12-25       5   
2         3          50         13  2025-01-26       4   
3         4          78         15  2025-04-21       3   
4         5          64          2  2023-07-16       3   

                             ReviewText  SentimentScore SentimentCategory  \
0  Average experience, nothing special.         -0.3089    Mixed Negative   
1            The quality is  top-notch.          0.0000          Positive   
2    Five stars for the quick delivery.          0.0000          Positive   
3   Good quality, but could be cheaper.          0.2382    Mixed Positive   
4  Average experience, nothing special.         -0.3089    Mixed Negative   

  SentimentBucket  
0    -0.49 to 0.0  
1     0.0 to 0.49  
2     0.0 to 0.49  
3     0.0 to 0.49  
4    -0.49 to 0.0  


In [43]:
# convertin the results into CSV
customer_reviews_df.to_csv('fact_customers_with_sentiment.csv', index=False)